# BatSimuPOP

# Simulation of population changes on bat colonies over time to assess the use of temporal sampling

# Input file:
1 Genepop file with 2 subpopulations (total number of individuals in genepop is inferior to the number of individuals in the simulation)

subpop 1: local colony that will undergo changes (additional info on age, age category and sex).

subpop 2: Rest of the population. Present to maintain gene flow over time. (additional info on sex and whether juvenile or adult)

Information on sex, age in years (for 1 subpop) or age category (juvenile-adult) is also available (file in fstat format 14 microsatellite loci + additional info)

In [1]:
from simuOpt import setOptions
setOptions(quiet=True)
import simuPOP as sim
from simuPOP.utils import importPopulation

pop = importPopulation('GENEPOP', 'Genpop_2subpops.txt')


Ther are 68 and 192 bats in two populations. No sex and age information yet.

In [2]:
# get age, agecat, sex from fstat file
%preview  pop_dump.txt --limit 500
import random

pop.addInfoFields(['age'])

# age_cat is not imported as they are defined by age 
# 0, 1, 2 as juvenile and 2-above as adult
with open('Fstat_dat(age,agecat,sex).txt') as fs:
    for idx, line in enumerate(fs):
        if idx < 10:
            # skip the first few lines
            continue
        if line.startswith('1 '):
            age, age_cat, sex = line.strip().split()[-3:]
            pop.individual(idx-18).setSex(sim.MALE if sex == '2' else sim.FEMALE )
            pop.individual(idx-18).age = int(age)
            #pop.individual(idx-18).age_cat = int(age_cat)
        elif line.startswith('2 '):
            age_cat, sex = line.strip().split()[-2:]
            pop.individual(idx-18).setSex(sim.MALE if sex == '2' else sim.FEMALE )
            # no age info
            pop.individual(idx-18).age = random.randint(0, 17)
            #pop.individual(idx-18).age_cat = int(age_cat)
sim.dump(pop, width=3, max=300, output='pop_dump.txt')     

In [3]:
pop.setVirtualSplitter(
    sim.CombinedSplitter([
        sim.SexSplitter(),
        sim.InfoSplitter(field='age', cutoff=[3] ),
    ]))

for x in range(0, 4):
    print(pop.subPopName([0, x]))
    print(pop.indInfo('age', subPop=[0, x]))

Male
()
Female
(2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 9.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 6.0, 1.0, 1.0, 12.0, 14.0, 13.0, 12.0, 9.0, 8.0, 6.0, 15.0, 15.0, 11.0, 13.0, 13.0, 12.0, 9.0, 2.0, 8.0, 10.0, 10.0, 7.0, 7.0, 8.0, 7.0, 7.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0)
age < 3
(2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0)
age >= 3
(3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 9.0, 6.0, 12.0, 14.0, 13.0, 12.0, 9.0, 8.0, 6.0, 15.0, 15.0, 11.0, 13.0, 13.0, 12.0, 9.0, 8.0, 10.0, 10.0, 7.0, 7.0, 8.0, 7.0, 7.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0)


# Life cycle of species to consider for overlapping generations:
Maxlifespan ~ 17 years old

minMatingAge ~ 2 years old

maxMatingAge ~ 17 years old

Number of young per year ~ 1

1 generation ~ 5 years

# Other settings:

Random mating in subpopulations

Maintain gene flow between both sub-populations using Migrator. This is sex-biased as females remain in the same colony and males disperse.

Female migration rate	0.001

Male migration rate	0.1

I think it might be better to start with the 68 females from the first population, then create an additional 62 males before the start of the simulation. This will come to a total number of 130 which is close to the effective population size I calculated for that population.

For the second population, you could create 110 females and 228 males before the start of the simulation as this would give an even sex ratio for the source population and a total of 550 individuals which is close to the whole effective population size.

In [4]:
# number of males and females?
print(pop.subPopSize([0,'Male']))
print(pop.subPopSize([0,'Female']))
print(pop.subPopSize([1,'Male']))
print(pop.subPopSize([1,'Female']))

# population 0, from 68 female to 68 female + 62 male
pop.resize([130, 338], propagate=True)
# set sex
for idx in range(68, 130):
    pop.individual(idx, 0).setSex(sim.MALE)
# add 110-37=73 male
for idx in range(192, 192+73):
    pop.individual(idx, 1).setSex(sim.MALE)
for idx in range(192+73, 338):
    pop.individual(idx, 1).setSex(sim.FEMALE) 
# number of males and females?
print(pop.subPopSize([0,'Male']))
print(pop.subPopSize([0,'Female']))
print(pop.subPopSize([1,'Male']))
print(pop.subPopSize([1,'Female']))

0
68
37
155
62
68
110
228


In [5]:
# as we are keeping subpopulation 1 as the "source" population,
# no migration from 0 to 1 is needed.

migr = sim.Migrator(
    rate=[
        [0, 0.1 ],
        [0, 0.001 ],
        [0.1, 0],
        [0.001, 0],
       ],
    mode = sim.BY_PROBABILITY,
    subPops=[(0, 'Male'), (0, 'Female'), (1, 'Male'), (1, 'Female')],
    toSubPops=[0,1],
)

# let us test the migration
pop.addInfoFields('migrate_to')

In [9]:
pop1 = pop.clone()
pop1.evolve(
    preOps=[
        sim.Stat(popSize=True,
             subPops=[(0, 0), (0, 1), (1, 0), (1, 1)],
             vars='popSize_sp'),
        sim.PyEval('''f"before migration: m/f 0: {subPop[(0,0)]['popSize']}/{subPop[(0,1)]['popSize']}''' + \
               ''' m/f 1: {subPop[(1,0)]['popSize']} {subPop[(1,1)]['popSize']}\\n"'''),
        migr,
        sim.Stat(popSize=True,
             subPops=[(0, 0), (0, 1), (1, 0), (1, 1)],
             vars='popSize_sp'),
        sim.PyEval('''f"after migration: m/f 0: {subPop[(0,0)]['popSize']}/{subPop[(0,1)]['popSize']}''' + \
               ''' m/f 1: {subPop[(1,0)]['popSize']} {subPop[(1,1)]['popSize']}\\n"'''),
    ],
    matingScheme=sim.RandomMating(), #subPopSize=[130, 338]),
    gen=5
)

before migration: m/f 0: 62/68 m/f 1: 110 228
after migration: m/f 0: 69/69 m/f 1: 103 227
before migration: m/f 0: 65/73 m/f 1: 167 163
after migration: m/f 0: 78/73 m/f 1: 154 163
before migration: m/f 0: 80/71 m/f 1: 146 171
after migration: m/f 0: 100/71 m/f 1: 126 171
before migration: m/f 0: 76/95 m/f 1: 149 148
after migration: m/f 0: 82/96 m/f 1: 143 147
before migration: m/f 0: 100/78 m/f 1: 141 149
after migration: m/f 0: 108/78 m/f 1: 133 149


5

# Example scenario:

Simulation of 20 generations (~200 years)

The initial population size is based on Ne estimates from the Genepop file (N genotypes < Actual population size), for example:

N0subpop1=130

N0subpop2=550

Subpop 2 remains constant over time

Subpop 1 undergoes an instant population change (InstantChangeModel) at generation 5 and declines to 80.

Then at generation 12 it undergoes a linear growth at rate 0.2 to a population of 150 (LinearGrowthModel) => Need of MultiStageModel 

# Outputs:

Need Genepop file outputs for each generation (or every 5 years) after breeding to include juveniles. The effective population size estimates will be done separately.

Output info on age and sex of individuals would also help with Ne estimates.